<a href="https://colab.research.google.com/github/aritraghsh09/GaMorNet/blob/master/tutorials/gamornet_tl_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab Stuff

Although this tutorial can be run on any machine which has GaMorNet installed, it's pretty handy to run this on Google Colab as you can easily use Colab's GPUs for this tutorial.

Note that with the free version of Colab, you will only have access to a limited amount of memory available. Thus, the number of images we use here for training/testing very small just for the purpose of demonstration. In reality, GaMorNet can hands hundreds of thousands of images. 

This first section is meant to be run only when following this tutorial in Google Colab


### Make things Fast!

Before we dive in, let's make sure we're using a GPU for this tutorial.  

To do this, select "Runtime" -> "Change runtime type" -> "Hardware accelerator" -> "GPU".

The following snippet will verify that we have access to a GPU.

In [1]:
import os
# Suppressing TF warnings and info for a cleaner environ
# Set this to 0,1 for info and warnings respectively.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
 
# Magic telling Colab we want TF version ~=1.0
%tensorflow_version 1.x

#Checking access to GPU
import tensorflow as tf
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

TensorFlow 1.x selected.
SUCCESS: Found GPU: /device:GPU:0


### Install GaMorNet

In [2]:
!pip install -q --upgrade gamornet

     |████████████████████████████████| 411.0MB 43kB/s 


In [3]:
##Checking which version of Tensorflow is being used and whether the installation worked.
import tensorflow as tf
print(tf.__version__)
from gamornet.keras_module import gamornet_train_keras, gamornet_tl_keras, gamornet_predict_keras
from gamornet.tflearn_module import gamornet_train_tflearn, gamornet_tl_tflearn, gamornet_predict_tflearn

1.15.2


Using TensorFlow backend.


# Reference

All mentions of "the paper" in this tutorial, refer to [Ghosh et. al. (2020)](https://iopscience.iop.org/article/10.3847/1538-4357/ab8a47)

# Installing Libraries Needed for this Tutorial

In [4]:
!pip install matplotlib

# Transfer Learning with GaMorNet

GaMorNet models that have been trained before can easily be refined using new data. This is the basic idea behind transfer learning.  

In this demonstration, we will start with simulation-trained SDSS model that we have released in the paper. Thereafter, we will perform transfer learning on this mode. For this, we will use 90 real SDSS galaxies for the purposes of training and 10 real SDSS galaxies for validation. All these images are in the g-band and are part of the SDSS data-set used in the paper. 

The simulations that the base model was trained on contain disk + bulge components. As described in the paper, we also convolved these simulations with a representative PSF and added representative noise. 

# Downloading the Data

First, let's download the images that we are going to use to transfer learn. We will download these into the local filesystem from Yale Astronomy's FTP service, where these are hosted.

We are going to download all the 100 images (90+10) as a single archive and then export it to a single folder called `training_imgs`. The iamges are in the FITS format and the names of the images are their SDSS Object IDs.

We are also going to download the `gal_para.txt` file containing the ground-truth parameters for the above galaxies from [Simard et. al. (2011)](https://iopscience.iop.org/article/10.1088/0067-0049/196/1/11). Using thes bulge-to-total light ratio of each galaxy, we will determine the labels to be used during the transfer learning process. 


*Tip: The `%%bash` command lets Colab know that all the commands in this shell needs to be passed the local unix virtual environment.*

*Tip: To view the files in use on Colab, click the folder icon on the left sidebar.*

In [5]:
%%bash
#get zip and txt file from server
wget ftp://ftp.astro.yale.edu/pub/aghosh/gamornet_tutorial_files/tl_images/tl_images.tar.gz
wget ftp://ftp.astro.yale.edu/pub/aghosh/gamornet_tutorial_files/tl_images/gal_para.txt

#Unzip the Archive
tar -xvf tl_images.tar.gz

./tl_images/587722953304440846-g.fits
./tl_images/587722981750014081-g.fits
./tl_images/587722982298026184-g.fits
./tl_images/587722982812155970-g.fits
./tl_images/587722982831161384-g.fits
./tl_images/587722983365279858-g.fits
./tl_images/587722983366721714-g.fits
./tl_images/587722983369408606-g.fits
./tl_images/587722983369408874-g.fits
./tl_images/587722983897890856-g.fits
./tl_images/587722983902806142-g.fits
./tl_images/587722983908049268-g.fits
./tl_images/587722983910932754-g.fits
./tl_images/587722983912767673-g.fits
./tl_images/587722984429322343-g.fits
./tl_images/587722984438038680-g.fits
./tl_images/587722984439545906-g.fits
./tl_images/587722984440659987-g.fits
./tl_images/587722984440791232-g.fits
./tl_images/587722984443216081-g.fits
./tl_images/587724197211144263-g.fits
./tl_images/587724197741854834-g.fits
./tl_images/587724197743362189-g.fits
./tl_images/587724197745918122-g.fits
./tl_images/587724197746704505-g.fits
./tl_images/587724197747032159-g.fits
./tl_images/

--2020-06-12 01:21:35--  ftp://ftp.astro.yale.edu/pub/aghosh/gamornet_tutorial_files/tl_images/tl_images.tar.gz
           => ‘tl_images.tar.gz’
Resolving ftp.astro.yale.edu (ftp.astro.yale.edu)... 128.36.139.12
Connecting to ftp.astro.yale.edu (ftp.astro.yale.edu)|128.36.139.12|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/aghosh/gamornet_tutorial_files/tl_images ... done.
==> SIZE tl_images.tar.gz ... 3709327
==> PASV ... done.    ==> RETR tl_images.tar.gz ... done.
Length: 3709327 (3.5M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  1%  585K 6s
    50K .......... .......... .......... .......... ..........  2% 25.6M 3s
   100K .......... .......... .......... .......... ..........  4% 1.65M 3s
   150K .......... .......... .......... .......... ..........  5%  298M 2s
   200K .......... .......... .......... .......... ..........  6% 2.08M 2s
   250K .......

# Preparing the Data

In this section, we will generate the training and validation image arrays as well as the corresponding labels to be used during the transfer learning process.


First, lets read in the `.txt` file, which will help us to figure out the label for each galaxy. 

In [0]:
import pylab as plt

#Let's read in the sim_para.txt file 
gal_para = plt.genfromtxt("./gal_para.txt",names=True,dtype=None,encoding=None)
 
#The file has a column called "bt_g" which is the bulge to total light ration
#for each galaxy as measured by Simard et. al. 2011. 

Next, let's define two convenience functions, which will assist us in creating the image and label arrays.

In [0]:
# Convenience Function to get and return images as numpy arrays

def image_handler(i):
  return np.reshape(fits.getdata("./tl_images/"+gal_para["file_name"][i],
                                 memmap=False),newshape=(167,167,1)) 
  #We use the reshape command just to add the extra 3rd dimension. The image is 
  #originally 167*167. So, in essence no re-sizing is taking place in the X or Y
  #directions.


# Convenience Function to get and return the training labels of each galaxy
# in the one-hot encoding format. i.e. disk-dominated galaxies will be represented
# by the array [1,0,0], bulge-dominated by [0,0,1] and indeterminate by [0,1,0]

def label_handler(i):
  
  target_vect = [0]*3

  if gal_para["bt_g"][i] < 0.45: #disk-dminated
    target_vect[0] = 1

  elif 0.45 <= gal_para["bt_g"][i] <= 0.55: #indeterminate
    target_vect[1] = 1

  else: #bulge-dominated
    target_vect[2] = 1

  return target_vect

Now, we are going to use the first 90 images to create the training set and the last 10 to create the validation set. We are multi-threading the process below -- although this is an absolute overkill for 100 images, it's very handy while dealing with large numbers of images. 

In [0]:
from multiprocessing import Pool
import numpy as np
from astropy.io import fits

NUM_THREADS = 2

pl = Pool(NUM_THREADS)
training_imgs = np.array(pl.map(image_handler,range(0,90)))
training_labels = np.array(pl.map(label_handler,range(0,90)))

valdiation_imgs = np.array(pl.map(image_handler,range(90,100)))
validation_labels = np.array(pl.map(label_handler,range(90,100)))

# Transfer Learning on GaMorNet using Keras

Now, we will be using the images and the labels generated above to transfer learn on GaMorNet. 

The main goal of transfer learning is to refine a previously trained model. Thus, as a starting point, we will be using the model trained on SDSS simulations in the paper. 

One of the other primary decisions to take during transfer learning is 

* whether you want to train all the layers or whether you want to freeze some of the them. The `trainable_bools` parameter controls this. 

* whether you want to initialize all the models from the previously trained model or whether you want to initialize some of the layers from scratch. The `load_layers_bools` parameter controls this. 



In [9]:
from gamornet.keras_module import gamornet_tl_keras

model = gamornet_tl_keras(training_imgs,training_labels,valdiation_imgs,validation_labels, 
                          input_shape='SDSS', load_layers_bools='load_bools_SDSS', 
                          trainable_bools='train_bools_SDSS', 
                          model_load_path='SDSS_sim', epochs=20, checkpoint_freq=10, 
                          batch_size=64, lr=0.00001, loss='categorical_crossentropy')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Fetching SDSS Sim Trained Weigths.....
Loading Layer0 from previous model.
Loading Layer3 from previous model.
Loading Layer6 from previous model.
Loading Layer7 from previous model.
Loading Layer8 from previous model.
Initializing Layer12 from scratch
Initializing Layer14 from scratch
Initializing Layer16 from scratch
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 90 samples, validate on 10 samples
Epoch 1/20
90/90 [==============================] - 7s 83ms/step - loss: 1.2314 - accuracy: 0.3222 - val_loss: 1.1959 - val_accuracy: 0.4000
Epoch 2/20
90/90 [==============================] - 0s 785us/step - loss: 1.1785 - accuracy: 0.3889 - val_loss: 1.1858 - val_accuracy: 0.4000
Epoch 3/20
90/90 [==============================] - 0s 775us/step - loss: 1.2381 - accuracy: 0.3444 - val_loss: 1.1722 - val_accuracy: 0.4000
Epoch 4/20
90/90 [============

In the output above, the `accuracy` and `loss` refer to the metrics calculated on the training set at the end of each epoch while `val_loss` and `val_accuracy` refer to the metrics calculated on the validation data. 

The above command trains the model using the images we prepared for 20 epochs using a learning rate of 0.00001 and a categorical cross-entropy loss function. The `checkpoint_freq = 10` parameter also ensures that every 10 epochs, a snapshot of the model is saved. These models are named as `model_x.hdf5` where x refers to the epoch at which the model was saved. The `input_shape` parameter specifies the shape of the input images. Setting this to `SDSS` automatically sets the value to `(167,167,1)`.

The value of the `load_layers_bools` should be an array of eight bools corresponding to Layer numbers 2, 5, 8, 9, 10, 13, 15, 17 of GaMorNet [(Schematic Diagram)](https://github.com/aritraghsh09/GaMorNet/blob/master/docs/source/images/gamornet_schematic_coloured.png). Setting each array element to `True` means that layer from the previous model. Setting this to `load_bools_SDSS` sets this to the configuration we used in the paper for SDSS data. 

The value of the `trainable_bools` should be an array of eight bools again corresponding to Layer numbers 2, 5, 8, 9, 10, 13, 15, 17 of GaMorNet [(Schematic Diagram)](https://github.com/aritraghsh09/GaMorNet/blob/master/docs/source/images/gamornet_schematic_coloured.png). Setting each array element to `True` means that wieghts and baises of that layer will be tuned during transfer learning. Otherwise, they would be kept frozen. Setting this to `train_bools_SDSS` sets this to the configuration we used in the paper for SDSS data. 

For an explanation of the different input parameters of `gamornet_tl_keras`, pelase have a look at the [API documentation](https://gamornet.readthedocs.io/en/latest/api_docs.html).

Thus, you can easily perform transfer learning using GaMorNet!! You can have a look at the model's structure using the command below. 

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 42, 42, 96)        11712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 96)        0         
_________________________________________________________________
local_response_normalization (None, 21, 21, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
local_response_normalization (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 384)      

**Important:**
The above process also generates a `metrics.csv` file, which contains the loss and accuracy calculated on the validation as well as the training data. 

We highly recommend using the data in this file to check how the loss and accuracies vary with training. This is extremely helpful in judging whether the model was trained properly and sufficiently. 